Functions to remove zeros from the data and plot the Nyquist diagrams:

In [10]:
import pandas as pd
import os
import plotly.graph_objects as go

def remove_zeros(file_path, create_new_file):
    df = pd.read_csv(file_path, sep='\s+', header=None, skiprows=1)

    df = df.apply(pd.to_numeric, errors='coerce')

    #Set column names
    df.columns = ["|Z|/Ohm", "freq/Hz", "Re(Z)/Ohm", "-Im(Z)/Ohm", "cycle number"]

    #Remove zeros
    df_filtered = df[(df[['|Z|/Ohm', 'freq/Hz', 'Re(Z)/Ohm', '-Im(Z)/Ohm', 'cycle number']] != 0).all(axis=1)]

    if create_new_file == True:
        base_name = os.path.basename(file_path)
        name, ext = os.path.splitext(base_name)
        output_file_name = f"{name}_clean{ext}"
        output_file_path = os.path.join(os.path.dirname(file_path), output_file_name)
        df_filtered.to_csv(output_file_path, sep='\t', index=False, header=False)

    column_headers = df.iloc[0]

    return df_filtered

def plot_spectrum(df, battery_name):
    cycle_column, re_column, im_column = 'cycle number', 'Re(Z)/Ohm', '-Im(Z)/Ohm'
    fig = go.Figure()
    
    cycles = df[cycle_column].unique()

    traces = []
    buttons = []

    for cycle in cycles:
        df_cycle = df[df[cycle_column] == cycle]
        trace = go.Scatter(x=df_cycle[re_column], y=df_cycle[im_column], mode='markers', name=f'Cycle {int(cycle)}', marker=dict(size=5.5))
        traces.append(trace)

    initial_visibility = [False] * len(cycles)
    for i, cycle in enumerate(cycles):
        button = dict(method='update',
                      label=f'Cycle {int(cycle)}',
                      visible=True,
                      args=[{'visible': [(not visibility) if j == i else visibility for j, visibility in enumerate(initial_visibility)]},
                            {'title': f'Impedance Plot'}])
        buttons.append(button)

    all_button = dict(method='update',label='All',visible=True,args=[{'visible': [True] * len(traces)}])
    buttons.append(all_button)

    fig.update_layout(autosize=False, width=700, height=600,
        updatemenus=[
            dict(
                type='buttons',
                direction='right',
                x=1.1,
                y=1.1,
                buttons=buttons
            )
        ],
        title=f'Impedance Plot of Battery {battery_name}',
        xaxis_title='Re(Z)/Ohm',
        yaxis_title='-Im(Z)/Ohm',
        showlegend=True,
        legend=dict(orientation='v', x=1.02, y=1)
    )

    for trace in traces:
        fig.add_trace(trace)

    fig.show()


Import all the EIS data and remove zeros:

In [11]:
file_paths = [
    ('Data/EIS_Data/NX001_2108_EIS.txt'),
    ('Data/EIS_Data/NX001_2401_EIS.txt'),
    ('Data/EIS_Data/NX002_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2108_EIS.txt'),
    ('Data/EIS_Data/NX006_2401_EIS.txt'),
    ('Data/EIS_Data/RS001_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2108_EIS.txt'),
    ('Data/EIS_Data/RS006_2401_EIS.txt'),
    ('Data/EIS_Data/SG003_2401_EIS.txt'),
    ('Data/EIS_Data/SG004_2401_EIS.txt')
]

clean_data = []

for file_path in file_paths:
    df = remove_zeros(file_path, False)
    battery_name = os.path.basename(file_path).split('_EIS.txt')[0]  # Extract battery name from file path
    clean_data.append((df, battery_name))  # Append tuple of DataFrame and battery name

df_NX001_2108, battery_name_NX001_2108 = clean_data[0]
df_NX001_2401, battery_name_NX001_2401 = clean_data[1]
df_NX002_2108, battery_name_NX002_2108 = clean_data[2]
df_NX006_2108, battery_name_NX006_2108 = clean_data[3]
df_NX006_2401, battery_name_NX006_2401 = clean_data[4]
df_RS001_2108, battery_name_RS001_2108 = clean_data[5]
df_RS006_2108, battery_name_RS006_2108 = clean_data[6]
df_RS006_2401, battery_name_RS006_2401 = clean_data[7]
df_SG003_2401, battery_name_SG003_2401 = clean_data[8]
df_SG004_2401, battery_name_SG004_2401 = clean_data[9]

Plot the Nyquist diagrams:

In [12]:
for df, battery_name in clean_data:  # Iterate over list of tuples
    plot_spectrum(df, battery_name)